In [ ]:
%cd /content/drive/MyDrive/CSE_497/Final

In [ ]:
import pandas as  pd

Unzipping the datasets-

The Kagle_Abstract_DS.zip can be downloaded from: https://www.kaggle.com/datasets/spsayakpaul/arxiv-paper-abstracts


The Research_Summarization.zip can be downloaded from: https://cs.stanford.edu/~myasu/projects/scisumm_net/

In [ ]:
!unzip Kaggle_Abstract_DS.zip
!unzip Research_Summarization.zip

We will combine our datasets in a single dataframe

In [ ]:
df = pd.DataFrame(columns=['problem', 'approach'])

In [ ]:
df1 = pd.read_csv('arxiv_data.csv')
df1.drop(columns=['terms'], inplace=True)
df1.rename(columns={'titles': 'problem', 'summaries': 'approach'}, inplace=True)
# ignoring short summaries/abstracts in an attempt that the model learns to generate detailed approaches
df1.drop(index=df1[df1['approach'].str.len() < 500].index, inplace=True)
df1.head()

The ScisummNet dataset has individual papers within separate directories. So, we need to iterate over them and collate the data.

In [ ]:
import os
import re

reg = re.compile(r'[A-Z][0-9]{2}-[0-9]{4}')
i = 0

df4 = pd.DataFrame(columns=['problem', 'approach'])
base = '/content/drive/MyDrive/CSE_497/Final/Data/top1000_complete'
for dir in os.listdir(base):
  file_name = os.path.join(base, dir, "summary", dir + ".gold.txt")
  with open(file_name, 'r') as f:
    data = f.readlines()
    df4.loc[i] = [data[0], "\n".join(data[1:])]
    i += 1
df4.head()

In [ ]:
# Saving in csv format such that it is easier to access later
df4.to_csv("scisumnett_data.csv")

In [ ]:
df4 = pd.read_csv("scisumnett_data.csv")
df4.drop(columns=['Unnamed: 0'], inplace=True)
df4.head()

In [ ]:
# Loading the CSV file generated by GPT 4.5
df6 = pd.read_csv("large_research_dataset.csv")
df6.rename(columns={'Research Problem': 'problem', 'Approach': 'approach'}, inplace=True)
df6.head()

In [ ]:
df1 = df1.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df6 = df6.reset_index(drop=True)

Modifying the titles and summaries to conform to the project requirements

In [ ]:
prefixes = ["Provide an approach for the following research problem: ",
            "Suggest a method to solve the following research challenge: ",
            "Devise a strategy to tackle the research problem: ",
            "How would you address the following research question? ",
            "What approach would you take for the following research task? ",
            "Propose a solution to the following research objective: ",
            "Explain how to work on the following research problem: ",
            "Generate a potential methodology for: ",
            "Design a research plan to address the problem: ",
            "What are possible steps to address the following issue? "]

approach_prefix = ["An effective approach would be to:",
                   "The proposed method involves the following steps:",
                   "To address this problem, one could:",
                   "The solution can be structured as follows:",
                   "A suitable approach might involve:",
                   "The following approch is based on a research paper:",
                   "To solve the problem, you can follow an approach as adopted in the following:"]

index = 0
def apply_prefixes(pre_list, text):
  global index
  index = (index + 1)%len(pre_list)
  return pre_list[index] + str(text)

df1['problem'] = df1['problem'].apply(lambda x: apply_prefixes(prefixes, x))

index = 0
df1['approach'] = df1['approach'].apply(lambda x: apply_prefixes(approach_prefix, x))

index = 0
df4['problem'] = df4['problem'].apply(lambda x: apply_prefixes(prefixes, x))

index = 0
df4['approach'] = df4['approach'].apply(lambda x: apply_prefixes(approach_prefix, x))

index = 0
df6['problem'] = df6['problem'].apply(lambda x: apply_prefixes(prefixes, x))

Saving the final dataset

In [ ]:
df = pd.concat([df, df1, df4, df6])
df.to_csv("final_raw_data.csv")